In [3]:
!pip install -q transformers
!pip install -q simpletransformers

In [4]:
import pandas as pd

df = pd.read_csv('data.csv')
train_df = df[df['split'] == 'train']
test_df = df[df['split'] == 'test']

Once done we can take a look at the `head()` of the training set to check if our data has been retrieved properly.

In [5]:
train_df.head()

,Unnamed: 0,acl.id,X1,text,label,type,target,level,split,round.base,annotator,round,acl.id.matched
0,1,acl1,1,dalits are lowlives,hate,notgiven,notgiven,original,train,1,GNZuCtwed3,1,NaN
1,2,acl10,2,gay people are a burden to our society,hate,notgiven,notgiven,original,train,1,GNZuCtwed3,1,NaN
2,3,acl100,3,Arabs are not welcome,hate,notgiven,notgiven,original,train,1,vDe7GN0NrL,1,NaN
3,4,acl1000,4,I'm not saying we should actually eliminate he...,hate,notgiven,notgiven,original,train,1,oemYWm1Tjg,1,NaN
14,15,acl1001,15,It's such a rape culture with these immigrants...,hate,notgiven,notgiven,original,train,1,CAgNlUizNm,1,NaN


In [6]:
train_df = train_df.replace({'label': {'hate': 1, 'nothate': 0}})
test_df = test_df.replace({'label': {'hate': 1, 'nothate': 0}})
train_df.head()

/tmp/ipython-input-3867108138.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df = train_df.replace({'label': {'hate': 1, 'nothate': 0}}) # relabel the `label` column, hate is 1 and nothate is 0
/tmp/ipython-input-3867108138.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_df = test_df.replace({'label': {'hate': 1, 'nothate': 0}}) # relabel the `label` column, hate is 1 and nothate is 0


,Unnamed: 0,acl.id,X1,text,label,type,target,level,split,round.base,annotator,round,acl.id.matched
0,1,acl1,1,dalits are lowlives,1,notgiven,notgiven,original,train,1,GNZuCtwed3,1,NaN
1,2,acl10,2,gay people are a burden to our society,1,notgiven,notgiven,original,train,1,GNZuCtwed3,1,NaN
2,3,acl100,3,Arabs are not welcome,1,notgiven,notgiven,original,train,1,vDe7GN0NrL,1,NaN
3,4,acl1000,4,I'm not saying we should actually eliminate he...,1,notgiven,notgiven,original,train,1,oemYWm1Tjg,1,NaN
14,15,acl1001,15,It's such a rape culture with these immigrants...,1,notgiven,notgiven,original,train,1,CAgNlUizNm,1,NaN


In [7]:
train_df = train_df.rename(columns={'label': 'labels'})
test_df = test_df.rename(columns={'label': 'labels'})

In [8]:
data = [[train_df.labels.value_counts()[0], test_df.labels.value_counts()[0]],
        [train_df.labels.value_counts()[1], test_df.labels.value_counts()[1]]]
pd.DataFrame(data, columns=['Train', 'Test'])

,Train,Test
0,15184,1852
1,17740,2268


In [9]:
train_args = {
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'sliding_window': True,
    'max_seq_length': 64,
    'num_train_epochs': 1,
    'train_batch_size': 128,
    'fp16': True,
    'output_dir': '/outputs/',
}

In [10]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn

logging.basicConfig(level=logging.DEBUG)
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.WARNING)

# We use the XLNet base cased pre-trained model.
model = ClassificationModel('roberta', 'roberta-base', num_labels=2, args=train_args)

model.train_model(train_df)

result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=sklearn.metrics.accuracy_score)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

Running Epoch 1 of 1:   0%|          | 0/298 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/simpletransformers/classification/classification_model.py:950: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


  0%|          | 0/4120 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/47 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/simpletransformers/classification/classification_model.py:1570: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


In [11]:
result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=sklearn.metrics.f1_score)

  0%|          | 0/4120 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/47 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/simpletransformers/classification/classification_model.py:1570: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


In [12]:
samples = [test_df[test_df['labels'] == 0].sample(1).iloc[0]['text']] # get a random sample from the test set which is nothate
predictions, _ = model.predict(samples)
label_dict = {0: 'nothate', 1: 'hate'}
for idx, sample in enumerate(samples):
  print('{} - {}: {}'.format(idx, label_dict[predictions[idx]], sample))

  0%|          | 0/1 [00:00<?, ?it/s]

Predicting:   0%|          | 0/1 [00:00<?, ?it/s]

0 - hate: Shiieeeet ma nigga! Fcking gotteem!


/usr/local/lib/python3.12/dist-packages/simpletransformers/classification/classification_model.py:2260: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


In [13]:
predictions, _ = model.predict(test_df['text'].tolist())
df = pd.DataFrame(predictions)
df.to_csv('results.txt', index=False, header=False) # saves the prediction results to a file in the colab environment

  0%|          | 0/4120 [00:00<?, ?it/s]

Predicting:   0%|          | 0/47 [00:00<?, ?it/s]

In [47]:
model.predict([["it is what it is"]])

  0%|          | 0/1 [00:00<?, ?it/s]

Predicting:   0%|          | 0/1 [00:00<?, ?it/s]

(array([0]), [array([[ 0.71630859, -0.71044922]])])

In [48]:
label_dict[0]

'nothate'